POKE :)

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from PIL import Image

## Statistics

In [2]:
stats = pd.read_csv('PokeDataset.csv')

In [3]:
stats['Type2'] = stats.Type2.fillna('')

In [4]:
conditions = [stats['Type2'] == '', stats['Type2'] != '']
values = [stats['Type1'], stats['Type1'] + '|' + stats['Type2']]

stats['Types'] = stats['Type1'] + ',' + stats['Type2']
stats['Types'] = stats['Types'].apply(lambda s: [l for l in str(s).split(',') if l not in [""]])
stats

,No,Name,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Color,Height,Weight,Types
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,Green,0.7,6.9,"[Grass, Poison]"
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,Green,1.0,13.0,"[Grass, Poison]"
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,Green,2.0,100.0,"[Grass, Poison]"
3,4,Charmander,Fire,,309,39,52,43,60,50,65,1,False,Red,0.6,8.5,[Fire]
4,5,Charmeleon,Fire,,405,58,64,58,80,65,80,1,False,Red,1.1,19.0,[Fire]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,901,Ursaluna,Ground,Normal,550,130,140,105,45,80,50,8,False,Brown,2.4,290.0,"[Ground, Normal]"
901,902,Basculegion,Water,Ghost,530,120,112,65,80,75,78,8,False,NaN,3.0,110.0,"[Water, Ghost]"
902,903,Sneasler,Fighting,Poison,510,80,130,60,40,80,120,8,False,Gray,1.3,43.0,"[Fighting, Poison]"
903,904,Overqwil,Dark,Poison,510,85,115,95,65,65,85,8,False,Black,2.5,60.5,"[Dark, Poison]"


## Popularity

In [5]:
ranks = pd.read_csv('PokeRank.csv')
ranks = ranks.rename(columns={"Nombre de votes": "Votes"})
ranks = ranks.rename(columns={"% de joueurs ayant voté pour": "Percentage"})

In [6]:
trad = pd.read_csv('PokeTrad.csv')

In [7]:
ranks['English'] = ranks['Pokémon'].map(trad.set_index('French')['English'])

In [8]:
ranks = ranks.dropna(axis=0)
total_votes = ranks['Votes'].sum() # après épurage des pokémons spéciaux
total_voters = 18004 # source Fildrong

In [9]:
ranks['Type1'] = ranks['English'].map(stats.set_index('Name')['Type1'])
ranks['Type2'] = ranks['English'].map(stats.set_index('Name')['Type2'])
ranks['Types'] = ranks['English'].map(stats.set_index('Name')['Types'])
ranks['Gen'] = ranks['English'].map(stats.set_index('Name')['Generation'])
ranks['Legendary'] = ranks['English'].map(stats.set_index('Name')['Legendary'])
ranks['Color'] = ranks['English'].map(stats.set_index('Name')['Color'])
ranks['No'] = ranks['English'].map(stats.set_index('Name')['No'])

In [10]:
ranks

,Classement,Pokémon,Votes,% de joueurs ayant vote pour,English,Type1,Type2,Types,Gen,Legendary,Color,No
0,1,Carchacrok,2190,"12,16%",Garchomp,Dragon,Ground,"[Dragon, Ground]",4,False,Blue,445
1,2,Ectoplasma,1999,"11,10%",Gengar,Ghost,Poison,"[Ghost, Poison]",1,False,Purple,94
2,3,Rayquaza,1672,"9,29%",Rayquaza,Dragon,Flying,"[Dragon, Flying]",3,True,Green,384
3,4,Amphinobi,1654,"9,19%",Greninja,Water,Dark,"[Water, Dark]",6,False,Blue,658
4,5,Lucario,1441,"8,00%",Lucario,Fighting,Steel,"[Fighting, Steel]",4,False,Blue,448
...,...,...,...,...,...,...,...,...,...,...,...,...
931,932,Muciole,11,"0,06%",Volbeat,Bug,,[Bug],3,False,Gray,313
932,933,Manglouton,10,"0,06%",Yungoos,Normal,,[Normal],7,False,Brown,734
933,934,Opermine,9,"0,05%",Binacle,Rock,Water,"[Rock, Water]",6,False,Brown,688
934,935,Noeunoeuf,8,"0,04%",Exeggcute,Grass,Psychic,"[Grass, Psychic]",1,False,Pink,102


In [11]:
LegVotes = ranks[ranks["Legendary"]==True]["Votes"].sum()
NonLegVotes = ranks[ranks["Legendary"]==False]["Votes"].sum()
total_votes = ranks["Votes"].sum()
Leg = ranks[ranks["Legendary"]==True]["Votes"].count()
NonLeg = ranks[ranks["Legendary"]==False]["Votes"].count()
total_pokemons = ranks["Votes"].count()

## VISUALIZATION
pie types votes</br>
~~pie gen votes~~</br>
true scatterpolar per type</br>
radar/scatterpolar stats pondérées
</br>bar vote just the curve

In [12]:
palette = {
    'Grass': '#2CA02C',
    'Fire': '#FD3216',
    'Water': '#0071FF',
    'Bug': '#90AD1C',
    'Normal': '#BAB0AC',
    'Poison': '#9E38AD',
    'Electric': '#EECA3B',
    'Ground': '#B68100',
    'Fairy': '#E49DCF',
    'Fighting': '#FF7818',
    'Psychic': '#F261A7',
    'Rock': '#A38B5B',
    'Ghost': '#4C3B7A',
    'Ice': '#19D3F3',
    'Dragon': '#1616A7',
    'Dark': '#222A2A',
    'Steel': '#778AAE',
    'Flying': '#72B7B2'
}

In [13]:
type_temp = stats['Types'].apply(lambda s: str(s).replace("'", "").replace('[', '').replace(']', '').split(', ')).explode().value_counts().sort_values(ascending=True)
color_temp = stats['Color'].value_counts().sort_values(ascending=True)

In [14]:
gen_columns = ['gen1', 'gen2', 'gen3', 'gen4', 'gen5', 'gen6', 'gen7', 'gen8']
type_freq = pd.DataFrame(columns=['type', 'freq']+gen_columns)
color_freq = pd.DataFrame(columns=['color', 'freq']+gen_columns)

In [15]:
for type in type_temp.index:
    type_freq = type_freq.append(pd.Series([type,
                                            type_temp[f'{type}'],
                                            stats[(stats['Generation']==1) & ((stats['Type1']==type) | (stats['Type2']==type))].count()[0], 
                                            stats[(stats['Generation']==2) & ((stats['Type1']==type) | (stats['Type2']==type))].count()[0],
                                            stats[(stats['Generation']==3) & ((stats['Type1']==type) | (stats['Type2']==type))].count()[0],
                                            stats[(stats['Generation']==4) & ((stats['Type1']==type) | (stats['Type2']==type))].count()[0],
                                            stats[(stats['Generation']==5) & ((stats['Type1']==type) | (stats['Type2']==type))].count()[0],
                                            stats[(stats['Generation']==6) & ((stats['Type1']==type) | (stats['Type2']==type))].count()[0],
                                            stats[(stats['Generation']==7) & ((stats['Type1']==type) | (stats['Type2']==type))].count()[0],
                                            stats[(stats['Generation']==8) & ((stats['Type1']==type) | (stats['Type2']==type))].count()[0],],
                                            index=['type', 'freq', 'gen1', 'gen2', 'gen3', 'gen4', 'gen5', 'gen6', 'gen7', 'gen8']),
                                            ignore_index=True)

for color in color_temp.index:
    color_freq = color_freq.append(pd.Series([color,
                                            color_temp[f'{color}'],
                                            stats[(stats['Generation']==1) & (stats['Color']==color)].count()[0], 
                                            stats[(stats['Generation']==2) & (stats['Color']==color)].count()[0],
                                            stats[(stats['Generation']==3) & (stats['Color']==color)].count()[0],
                                            stats[(stats['Generation']==4) & (stats['Color']==color)].count()[0],
                                            stats[(stats['Generation']==5) & (stats['Color']==color)].count()[0],
                                            stats[(stats['Generation']==6) & (stats['Color']==color)].count()[0],
                                            stats[(stats['Generation']==7) & (stats['Color']==color)].count()[0],
                                            stats[(stats['Generation']==8) & (stats['Color']==color)].count()[0],],
                                            index=['color', 'freq', 'gen1', 'gen2', 'gen3', 'gen4', 'gen5', 'gen6', 'gen7', 'gen8']),
                                            ignore_index=True)


C:\Windows\Temp\ipykernel_15788\2656124746.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  type_freq = type_freq.append(pd.Series([type,
C:\Windows\Temp\ipykernel_15788\2656124746.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  type_freq = type_freq.append(pd.Series([type,
C:\Windows\Temp\ipykernel_15788\2656124746.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  type_freq = type_freq.append(pd.Series([type,
C:\Windows\Temp\ipykernel_15788\2656124746.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  type_freq = type_freq.append(pd.Series([type,
C:\Windows\Temp\ipykernel_15788\2656124746.py:2: FutureWarning: The frame.append

### OLD BAR CHART

In [16]:
fig = px.bar(type_freq,
             x='freq',
             y='type',
             color='type',
             orientation="h",
             color_discrete_map=palette,
             text='freq',
             hover_data={'type':False,
                         'freq':False,
                         'gen1':True,
                         'gen2':True,
                         'gen3':True,
                         'gen4':True,
                         'gen5':True,
                         'gen6':True,
                         'gen7':True,
                         'gen8':True,},
             labels={'x': 'Number of Pokémons per type', 'y': ""})

fig.update_layout(height=700, width=1200, showlegend=False, yaxis={'visible': False}, xaxis={'title': 'Pokémon type frequency accross all Generations'})

sources = [f"types\{typ}.png" for typ in type_freq['type'].unique()]

count=0
for col, src in enumerate(sources):
    fig.add_layout_image(
        # row=1,
        # col=col + 1,
        source=Image.open(sources[col]),
        xref="x domain",
        yref="y domain",
        x=0,
        y=0.985-col*0.0555,
        xanchor="right",
        yanchor="top",
        sizex=0.06,
        sizey=0.06
    ) 

fig.show()

In [17]:
# fig = px.bar(color_freq,
#              x='freq',
#              y='color',
#              color='color',
#              orientation="h",
#              color_discrete_map={'Black':'black',
#                                 'Pink':'pink',
#                                 'Yellow':'yellow',
#                                 'White':'white',
#                                 'Purple':'purple',
#                                 'Red':'red',
#                                 'Gray':'darkgray',
#                                 'Green':'green',
#                                 'Brown':'brown',
#                                 'Blue':'blue'},
#              text='freq',
#              hover_data={'color':False,
#                          'freq':False,
#                          'gen1':True,
#                          'gen2':True,
#                          'gen3':True,
#                          'gen4':True,
#                          'gen5':True,
#                          'gen6':True,
#                          'gen7':True,
#                          'gen8':True,},
#              labels={'x': 'Number of Pokémons per color', 'y': ""})

# fig.update_layout(height=700, showlegend=False, yaxis={'visible': False}, xaxis={'title': 'Pokémon color frequency accross all Generations'})

# fig.show()

In [18]:
cols=['HP', 'Attack', 'Defense', 'Speed', 'Sp. Def', 'Sp. Atk']

fig = make_subplots(rows=3, cols=6, start_cell="top-left", #subplot_titles=stats['Type1'].unique(),
                     specs=[[{"type": "polar"}, {"type": "polar"}, {"type": "polar"}, {"type": "polar"}, {"type": "polar"}, {"type": "polar"}],
                            [{"type": "polar"}, {"type": "polar"}, {"type": "polar"}, {"type": "polar"}, {"type": "polar"}, {"type": "polar"}],
                            [{"type": "polar"}, {"type": "polar"}, {"type": "polar"}, {"type": "polar"}, {"type": "polar"}, {"type": "polar"}]])

Nrow, Ncol = 1, 1
colormap = {item : palette.get(type) for item in cols}

total = stats[cols].describe().transpose()
total = total.append(total.iloc[0])

for type in stats['Type1'].unique():
    df = stats[(stats['Type1']==type) | (stats['Type2']==type)]
    statistics = df[cols].describe().transpose()
    statistics = statistics.append(statistics.iloc[0])
    statistics = statistics.rename({'Attack': 'Atk', 'Defense':'Def'})
    average = df['Total'].mean().round(decimals=1)

    fig.add_trace(go.Scatterpolar(
                        r=statistics['mean'],
                        theta=statistics.index, 
                        line={'color':palette.get(type)},
                        fillcolor=palette.get(type),
                        fill='toself',
                        name=average,
                        marker={'size':0.1},
                        hovertemplate="%{r:.1f}<extra></extra>"
                    ), row=Nrow, col=Ncol)

    # GLOBAL AVERAGE
    fig.add_trace(go.Scatterpolar(
                        r=total['mean'],
                        theta=statistics.index, 
                        line={'dash':'dash','color':'black'},
                        # marker={'size':0.1},
                        # hovertemplate="%{r:.1f}<extra></extra>"
                    ), row=Nrow, col=Ncol)

    # STD UP
    # fig.add_trace(go.Scatterpolar(
    #                     r=statistics['mean']+statistics['std'],
    #                     theta=statistics.index, 
    #                     line={'dash':'dash','color':'black'},
    #                     # marker={'size':0.1},
    #                     # hovertemplate="%{r:.1f}<extra></extra>"
    #                 ), row=Nrow, col=Ncol)

    # STD DOWN
    # fig.add_trace(go.Scatterpolar(
    #                     r=statistics['mean']-statistics['std'],
    #                     theta=statistics.index, 
    #                     line={'dash':'dash','color':'black'},
    #                     # marker={'size':0.1},
    #                     # hovertemplate="%{r:.1f}<extra></extra>"
    #                 ), row=Nrow, col=Ncol)

    # 1st QUARTILE
    # fig.add_trace(go.Scatterpolar(
    #                     r=statistics['25%'],
    #                     theta=statistics.index, 
    #                     line={'dash':'dash','color':'black'},
    #                     # marker={'size':0.1},
    #                     # hovertemplate="%{r:.1f}<extra></extra>"
    #                 ), row=Nrow, col=Ncol)

    # 3rd QUARTILE
    # fig.add_trace(go.Scatterpolar(
    #                     r=statistics['75%'],
    #                     theta=statistics.index, 
    #                     line={'dash':'dash','color':'black'},
    #                     # marker={'size':0.1},
    #                     # hovertemplate="%{r:.1f}<extra></extra>"
    #                 ), row=Nrow, col=Ncol)

    if Ncol!=6:
        Ncol+=1
    else:
        Ncol=1
        Nrow+=1

sources = [f"types\{typ}.png" for typ in stats['Type1'].unique()]

count=0
for col, src in enumerate(sources):
    fig.add_layout_image(
        # row=1,
        # col=col + 1,
        source=Image.open(sources[col]),
        xref="x domain",
        yref="y domain",
        x=0.052+(col%6)*0.172,
        y=1.05-count*0.36,
        xanchor="right",
        yanchor="top",
        sizex=0.1,
        sizey=0.1,
    )
    if col==5 or col==11:
        count+=1

fig.update_layout(height=700, width=1500, showlegend=False)
fig.update_polars(gridshape='linear', 
                  angularaxis=dict(
                       thetaunit="degrees",
                       rotation=90,
                      direction='clockwise'),
                  radialaxis=dict(visible=False,range=[0, 110]))
fig.show()

C:\Windows\Temp\ipykernel_15788\362125233.py:12: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Windows\Temp\ipykernel_15788\362125233.py:17: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Windows\Temp\ipykernel_15788\362125233.py:17: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Windows\Temp\ipykernel_15788\362125233.py:17: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Windows\Temp\ipykernel_15788\362125233.py:17: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Windows\Temp\ipykernel_15788\362125233.py:17: FutureWarning:

The frame.appen

In [19]:
stat = {}
temp = 0
color_count = {
    'Black':0,
    'Pink':0,
    'Yellow':0,
    'White':0,
    'Purple':0,
    'Red':0,
    'Gray':0,
    'Green':0,
    'Brown':0,
    'Blue':0
}
type_count = {
    'Grass': 0,
    'Fire': 0,
    'Water': 0,
    'Bug': 0,
    'Normal': 0,
    'Poison': 0,
    'Electric': 0,
    'Ground': 0,
    'Fairy': 0,
    'Fighting': 0,
    'Psychic': 0,
    'Rock': 0,
    'Ghost': 0,
    'Ice': 0,
    'Dragon': 0,
    'Dark': 0,
    'Steel': 0,
    'Flying': 0,
    '': 0
}

for statistique in cols:
    for i in range(898):
        temp += int(stats.iloc[i][f'{statistique}']) * int(ranks[ranks['English']==stats.iloc[i]['Name']]['Votes'])
        if statistique == "HP":
            color_count[str(ranks[ranks['English']==stats.iloc[i]['Name']]['Color'].values[0])] += int(ranks[ranks['English']==stats.iloc[i]['Name']]['Votes'])
            type_count[str(ranks[ranks['English']==stats.iloc[i]['Name']]['Type1'].values[0])] += int(ranks[ranks['English']==stats.iloc[i]['Name']]['Votes'])
            type_count[str(ranks[ranks['English']==stats.iloc[i]['Name']]['Type2'].values[0])] += int(ranks[ranks['English']==stats.iloc[i]['Name']]['Votes'])
    stat[statistique]=(temp/total_votes).round(decimals=2)
    temp = 0

In [20]:
color_count = pd.DataFrame.from_dict(color_count, columns=['count'], orient='index').sort_values(by="count").reset_index().rename(columns={'index':'color'})
type_count = pd.DataFrame.from_dict(type_count, columns=['count'], orient='index').sort_values(by="count").reset_index().rename(columns={'index':'type'})#.drop(18,axis=0)

In [47]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(y=color_freq['freq'],
                     x=color_freq['color'],
                     marker_color=['black',
                                'pink',
                                'yellow',
                                'white',
                                'purple',
                                'red',
                                'darkgray',
                                'green',
                                'brown',
                                'blue'],
                     yaxis='y2',
                     offsetgroup=2,
                     text=color_freq['freq'],
                     textposition='outside',
                     customdata=color_freq[gen_columns],
                     hovertemplate="</br>1G: %{customdata[0]}</br>" +
                                   "2G: %{customdata[1]}</br>" +
                                   "3G: %{customdata[2]}</br>" +
                                   "4G: %{customdata[3]}</br>" +
                                   "5G: %{customdata[4]}</br>" +
                                   "6G: %{customdata[5]}</br>" +
                                   "7G: %{customdata[6]}</br>" +
                                   "8G: %{customdata[7]}</br>" +
                                   "<extra></extra>"
                    ))

fig.add_trace(go.Bar(y=color_count['count'],
                     x=color_count['color'],
                     marker={"pattern_fillmode":"replace",
                             "line":{"width":1},
                             "line_color":['pink',
                                'black',
                                'darkgray',
                                'white',
                                'yellow',
                                'purple',
                                'red',
                                'brown',
                                'green',
                                'blue']},
                     marker_pattern_shape=".",
                     marker_pattern_size=0.5,
                     marker_color=['pink',
                                'black',
                                'darkgray',
                                'white',
                                'yellow',
                                'purple',
                                'red',
                                'brown',
                                'green',
                                'blue'],
                     yaxis='y',
                     offsetgroup=1,
                     text=color_count['count'],
                     textposition='outside',
                     hovertemplate="%{y} votes<extra></extra>"
                     ))

fig.update_layout(height=700,
                  yaxis={'title':'Nombre de votes par couleur de Pokémon', 'range':[0,31000]},
                  yaxis2={'title':'Nombre de Pokémons par couleur', 'range':[0,200]},
                  showlegend=False,
                  barmode='group',
                  xaxis={'title': 'Pokémon color frequency accross all Generations'})
fig.show()

In [49]:
alternate_palette=['#19D3F3','#4C3B7A','#778AAE','#E49DCF','#222A2A','#1616A7','#EECA3B','#FF7818','#A38B5B','#B68100','#FD3216','#9E38AD','#90AD1C','#F261A7','#72B7B2','#2CA02C','#BAB0AC','#0071FF']
alternate_palette2=['#19D3F3','#A38B5B','#90AD1C','#E49DCF','#9E38AD','#EECA3B','#BAB0AC','#FF7818','#4C3B7A','#778AAE','#B68100','#F261A7','#2CA02C','#222A2A','#FD3216','#1616A7','#72B7B2','#0071FF']

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(y=type_freq['freq'],
                     x=type_freq['type'],
                     marker_color=alternate_palette,
                    yaxis='y2',
                     offsetgroup=2,
                     text=type_freq['freq'],
                     textposition='outside',
                     customdata=type_freq[gen_columns],
                     hovertemplate="</br>1G: %{customdata[0]}</br>" +
                                   "2G: %{customdata[1]}</br>" +
                                   "3G: %{customdata[2]}</br>" +
                                   "4G: %{customdata[3]}</br>" +
                                   "5G: %{customdata[4]}</br>" +
                                   "6G: %{customdata[5]}</br>" +
                                   "7G: %{customdata[6]}</br>" +
                                   "8G: %{customdata[7]}</br>" +
                                   "<extra></extra>"
                    ))

fig.add_trace(go.Bar(y=type_count.iloc[0:18]['count'],
                     x=type_count.iloc[0:18]['type'],
                     marker={"pattern_fillmode":"replace",
                             "line":{"width":1},
                             "line_color":alternate_palette2},
                     marker_pattern_shape=".",
                     marker_pattern_size=0.5,
                     marker_color=alternate_palette2,
                     yaxis='y',
                     offsetgroup=1,
                     text=type_count.iloc[0:18]['count'],
                     hovertemplate="%{y} votes<extra></extra>"
                     ))

fig.update_layout(height=700,
                  yaxis={'title':'Nombre de votes par couleur de Pokémon', 'range':[0,21000]},
                  yaxis2={'title':'Nombre de Pokémons par couleur', 'range':[0,150]},
                  showlegend=False,
                  barmode='group',
                  xaxis={'title': 'Pokémon color frequency accross all Generations'})
fig.show()

In [183]:
alternate_palette=['#19D3F3','#4C3B7A','#778AAE','#E49DCF','#222A2A','#1616A7','#EECA3B','#FF7818','#A38B5B','#B68100','#FD3216','#9E38AD','#90AD1C','#F261A7','#72B7B2','#2CA02C','#BAB0AC','#0071FF']
alternate_palette2=['#19D3F3','#A38B5B','#90AD1C','#E49DCF','#9E38AD','#EECA3B','#BAB0AC','#FF7818','#4C3B7A','#778AAE','#B68100','#F261A7','#2CA02C','#222A2A','#FD3216','#1616A7','#72B7B2','#0071FF']
from plotly.graph_objs.layout import XAxis
fig = go.Figure(data=[
    go.Bar(
        x=type_freq['freq'],
        y=type_freq['type'],
        marker_color=alternate_palette,
        xaxis='x2',
        offsetgroup=2,
        text=type_freq['freq'],
        textposition='inside',
        orientation='h',
        customdata=type_freq[gen_columns],
        hovertemplate="</br>1G: %{customdata[0]}</br>" +
                    "2G: %{customdata[1]}</br>" +
                    "3G: %{customdata[2]}</br>" +
                    "4G: %{customdata[3]}</br>" +
                    "5G: %{customdata[4]}</br>" +
                    "6G: %{customdata[5]}</br>" +
                    "7G: %{customdata[6]}</br>" +
                    "8G: %{customdata[7]}</br>" +
                    "<extra></extra>"
        ),
    go.Bar(
        x=type_count.iloc[0:18]['count'],
        y=type_count.iloc[0:18]['type'],
        marker={"pattern_fillmode":"replace",
                "line":{"width":1},
                "line_color":alternate_palette2},
        marker_pattern_shape=".",
        marker_pattern_size=1,
        marker_color=alternate_palette2,
        textposition='inside',
        orientation='h',
        xaxis='x',
        offsetgroup=1,
        text=type_count.iloc[0:18]['count'],
        hovertemplate="%{y} votes<extra></extra>"
        ),
])



layout = go.Layout(
    xaxis2 = XAxis( 
        overlaying='x',
        side='top',
    ),
    barmode='group',
    )

fig.layout = layout

fig.update_layout(height=1000,
                  xaxis={'title':{'text':'Nombre de votes par couleur de Pokémon'}, 'range':[0,21000]},
                  xaxis2={'title':'Nombre de Pokémons par couleur', 'range':[0,150]},
                  showlegend=False,
                  title="Pokémon color frequency accross all GenerationsPlot Title",
                  margin={'t':150}
                )
fig.update_yaxes(showticklabels=False) # Hide y axis ticks

fig.add_vline(x=int(type_count.iloc[0:18]['count'].mean()), line_width=1, line_dash="dash", line_color="red")

sources_types = ['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric', 'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice', 'Dragon', 'Dark', 'Steel', 'Flying']
images = dict(zip(sources_types, sources))

for i, type in enumerate(type_freq['type']):
    fig.add_layout_image(
        source=Image.open(images[type]),
        xref="paper",
        yref="paper",
        x=0.01,
        y=0.012+i*0.0556,
        xanchor="right",
        yanchor="bottom",
        sizex=0.1,
        sizey=0.1
    ) 
    
fig.show()

In [166]:
sources_types = ['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric', 'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice', 'Dragon', 'Dark', 'Steel', 'Flying']
images = dict(zip(sources_types, sources))
images

{'Grass': 'types\\Grass.png',
 'Fire': 'types\\Fire.png',
 'Water': 'types\\Water.png',
 'Bug': 'types\\Bug.png',
 'Normal': 'types\\Normal.png',
 'Poison': 'types\\Poison.png',
 'Electric': 'types\\Electric.png',
 'Ground': 'types\\Ground.png',
 'Fairy': 'types\\Fairy.png',
 'Fighting': 'types\\Fighting.png',
 'Psychic': 'types\\Psychic.png',
 'Rock': 'types\\Rock.png',
 'Ghost': 'types\\Ghost.png',
 'Ice': 'types\\Ice.png',
 'Dragon': 'types\\Dragon.png',
 'Dark': 'types\\Dark.png',
 'Steel': 'types\\Steel.png',
 'Flying': 'types\\Flying.png'}

In [36]:
fig = make_subplots(rows=1, cols=2, subplot_titles=["Votes", "Pokédex"], specs=[[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(values=[NonLegVotes, LegVotes], labels=['Légendaires', 'Non Légendaires'], opacity=0.9, hovertemplate="<extra></extra>", name="Votes"), row=1, col=1)
fig.add_trace(go.Pie(values=[NonLeg, Leg], labels=['Légendaires', 'Non Légendaires'], opacity=0.9, hovertemplate="<extra></extra>", name="Pokédex"), row=1, col=2)
fig.update_traces(hole=.4, textfont_size=20,
                  marker=dict(colors=["lightgray", "darkblue"], line=dict(color='#000000', width=2)))
fig.update_layout(
    annotations=[dict(text='Votes', x=0.25, y=0.36, font_size=20, showarrow=False,bordercolor="#c7c7c7",borderwidth=2,borderpad=4,bgcolor='rgba(255,255,255,0.85)'),
                 dict(text='Pokédex', x=0.75, y=0.36, font_size=20, showarrow=False,bordercolor="#c7c7c7",borderwidth=2,borderpad=4,bgcolor='rgba(255,255,255,0.85)')])

img_width = 1024
img_height = 575
scale_factor = 0.85

fig.update_xaxes(
    visible=False,
    range=[0, img_width * scale_factor]
)

fig.update_yaxes(
    visible=False,
    range=[0, img_height * scale_factor],
    # the scaleanchor attribute ensures that the aspect ratio stays constant
    scaleanchor="x"
)

fig.add_layout_image(
        dict(
            source="legendary.png",
            x=0,
            sizex=img_width * scale_factor,
            y=img_height * scale_factor,
            sizey=img_height * scale_factor,
            xref="x",
            yref="y",
            opacity=0.65,
            layer="below",
            sizing="stretch"
        )
)
fig.data[0].domain={'x': [0, 0.5], 'y': [0.1, 0.7]}
fig.data[1].domain={'x': [0.5, 1], 'y': [0.1, 0.7]}

fig.update_layout(
    width=img_width * scale_factor,
    height=img_height * scale_factor,
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
    legend=dict(yanchor="top", y=0.9, xanchor="left", x=0.35, title='LeS PoKéMoNs LéGeNdAiReS sOnT TrOp StYléS',bordercolor="#c7c7c7",
        borderwidth=2,
        bgcolor='rgba(255,255,255,0.85)')
)
fig.show()

In [38]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(x=ranks['Gen'], y=ranks['Votes'], customdata=ranks['Pokémon'], hovertemplate="%{customdata}<extra></extra>", showlegend = False),
    secondary_y=False,
)
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=0.5, opacity=0.8)

y = [stats[stats['Generation']==gen].count()[0] for gen in stats['Generation'].unique()]

fig.add_trace(
    go.Scatter(x=stats['Generation'].unique(), y=y, hovertemplate="Il y a %{y} Pokémons dans la Génération %{x}<extra></extra>", showlegend = False),
    secondary_y=True,
)

fig.update_layout(title='Nombre de votes vs nombre de Pokémons par génération')
fig.update_yaxes(range=[0,200], title_text="Nombre de Pokémons par génération", secondary_y=True)
fig.update_yaxes(title_text="Nombre de Votes par génération", secondary_y=False)
fig.show()

In [25]:
stats.head()

,No,Name,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Color,Height,Weight,Types
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,Green,0.7,6.9,"[Grass, Poison]"
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,Green,1.0,13.0,"[Grass, Poison]"
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,Green,2.0,100.0,"[Grass, Poison]"
3,4,Charmander,Fire,,309,39,52,43,60,50,65,1,False,Red,0.6,8.5,[Fire]
4,5,Charmeleon,Fire,,405,58,64,58,80,65,80,1,False,Red,1.1,19.0,[Fire]


In [26]:
ranks.head()

,Classement,Pokémon,Votes,% de joueurs ayant vote pour,English,Type1,Type2,Types,Gen,Legendary,Color,No
0,1,Carchacrok,2190,"12,16%",Garchomp,Dragon,Ground,"[Dragon, Ground]",4,False,Blue,445
1,2,Ectoplasma,1999,"11,10%",Gengar,Ghost,Poison,"[Ghost, Poison]",1,False,Purple,94
2,3,Rayquaza,1672,"9,29%",Rayquaza,Dragon,Flying,"[Dragon, Flying]",3,True,Green,384
3,4,Amphinobi,1654,"9,19%",Greninja,Water,Dark,"[Water, Dark]",6,False,Blue,658
4,5,Lucario,1441,"8,00%",Lucario,Fighting,Steel,"[Fighting, Steel]",4,False,Blue,448
